# Подготовкка RAG системы

Приступим к настройке RAG системы

In [5]:
from llama_index.llms.ollama import Ollama

from llama_parse import LlamaParse
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, PromptTemplate
from llama_index.core.embeddings import resolve_embed_model

# import nest_asyncio

from dotenv import load_dotenv
load_dotenv()

True

In [1]:
! ollama list

NAME               ID              SIZE      MODIFIED    
llama3.2:latest    a80c4f17acd5    2.0 GB    2 weeks ago    


Теперь мы можем релизовтаь простой поисковик, который будет переводить документы в векторный формат который позвлит нам производить поиск по данным хранящимся в файле

Попробуем реализовать бейзлайн RAG приложения

In [6]:
# В первый раз ставим время побольше, чтобы модель установилась
llm = Ollama(model="llama3.2:latest", request_timeout=120.0)

# llm.complete('Hello. How are you?')

# Напишем запрос к модели
prompt = """
Hello. Tell us what kind of model you are.
"""
mesage = llm.complete(prompt)

print(mesage.text)

Hello! I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI." It's a type of sequence-to-sequence model that uses self-attention mechanisms to process and generate human-like language. My primary function is to understand and respond to natural language inputs, providing information and assistance on a wide range of topics. I'm constantly learning and improving my responses based on user interactions like this one. How can I help you today?


LlamaParse — это универсальный инструмент для подготовки данных для больших языковых моделей.

[Основная цель LlamaParse — парсить и очищать ваши данные, гарантируя их хорошее качество перед передачей в любой последующий сценарий использования LLM, например, расширенный RAG.](https://docs.llamaindex.ai/en/stable/llama_cloud/llama_parse/)

[Introducing LlamaCloud and LlamaParse](https://www.llamaindex.ai/blog/introducing-llamacloud-and-llamaparse-af8cedf9006b)

In [10]:
import nest_asyncio
nest_asyncio.apply()


parser = LlamaParse(result_type='markdown')

file_extractor = {'.pdf': parser}

# Смотрим директорию и ищем файлы в формате pdf
documents = SimpleDirectoryReader(input_dir='./data', file_extractor=file_extractor).load_data()

Started parsing the file under job_id be9c8b77-4c2d-448e-982b-8a5837b01f0c
...

In [11]:
# Грузим модель для создания векторных эмбеддингов текста
embed_model = resolve_embed_model('local:BAAI/bge-m3')
# Создаём индекс векторов для эффективного поиска по коллекции документов
vector_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
# Создаём "движок запросов" на основе созданного индекса векторов
query_engine = vector_index.as_query_engine(llm=llm)

c:\Users\Asus\Documents\GitHub\llamaindex_RAG_MLLM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
result = query_engine.query('Based on the existing table of contents, tell us what the existing work is about.')
print(result)

The existing work appears to be a grammar guide or textbook, covering various aspects of writing and language usage. The topics listed in the table of contents suggest that it covers subjects such as sentence structure, punctuation, verb tenses, and word choice, with a focus on clarity and effectiveness.


Теперь попробуем написать свой локальный поисковик

# Запуск RAG локально (Подготовка)

In [18]:
# Установка необходимых зависимостей
# ! pip install weaviate-client sentence-transformers PyPDF2 sqlite3

In [ ]:
# Запуск weaviate - локально
# Для этого отдельно настроим ollama и weaviate

### Проверка готовкности

#### Проверка работоспособности weaviate

In [1]:
!curl -X GET http://127.0.0.1:8083/v1/

{"links":[{"href":"/v1/meta","name":"Meta information about this instance/cluster"},{"documentationHref":"https://weaviate.io/developers/weaviate/api/rest#tag/schema/get/schema","href":"/v1/schema","name":"view complete schema"},{"documentationHref":"https://weaviate.io/developers/weaviate/api/rest#tag/schema/put/schema/{className}","href":"/v1/schema{/:className}","name":"CRUD schema"},{"documentationHref":"https://weaviate.io/developers/weaviate/api/rest#tag/objects/","href":"/v1/objects{/:id}","name":"CRUD objects"},{"documentationHref":"https://weaviate.io/developers/weaviate/api/rest#tag/classifications","href":"/v1/classifications{/:id}","name":"trigger and view status of classifications"},{"documentationHref":"https://weaviate.io/developers/weaviate/api/rest#tag/well-known/get/.well-known/live","href":"/v1/.well-known/live","name":"check if Weaviate is live (returns 200 on GET when live)"},{"documentationHref":"https://weaviate.io/developers/weaviate/api/rest#tag/well-known/get/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1383  100  1383    0     0   291k      0 --:--:-- --:--:-- --:--:--  337k


#### Проверка работоспособности ollama

In [2]:
!curl -X GET http://localhost:11437/api/version

{"version":"0.3.14"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    20  100    20    0     0   5740      0 --:--:-- --:--:-- --:--:--  6666


#### Загрузка модели [qwen2.5](https://github.com/QwenLM/Qwen2.5?ysclid=m5fieixax7580998174)

In [3]:
! curl http://localhost:11437/api/pull -d "{\"name\": \"qwen2.5:0.5b\"}"

{"status":"pulling manifest"}
{"status":"pulling c5396e06af29","digest":"sha256:c5396e06af294bd101b30dce59131a76d2b773e76950acc870eda801d3ab0515","total":397807936,"completed":397807936}
{"status":"pulling 66b9ea09bd5b","digest":"sha256:66b9ea09bd5b7099cbb4fc820f31b575c0366fa439b08245566692c6784e281e","total":68,"completed":68}
{"status":"pulling eb4402837c78","digest":"sha256:eb4402837c7829a690fa845de4d7f3fd842c2adee476d5341da8a46ea9255175","total":1482,"completed":1482}
{"status":"pulling 832dd9e00a68","digest":"sha256:832dd9e00a68dd83b3c3fb9f5588dad7dcf337a0db50f7d9483f310cd292e92e","total":11343,"completed":11343}
{"status":"pulling 005f95c74751","digest":"sha256:005f95c7475154a17e84b85cd497949d6dd2a4f9d77c096e3c66e4d9c32acaf5","total":490,"completed":490}
{"status":"verifying sha256 digest"}
{"status":"writing manifest"}
{"status":"success"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    54    0    30  100    24    934    747 --:--:-- --:--:-- --:--:--  1636
100    54    0    30  100    24     24     19  0:00:01  0:00:01 --:--:--    44
100    54    0    30  100    24     13     10  0:00:02  0:00:02 --:--:--    24
100    54    0    30  100    24      9      7  0:00:03  0:00:03 --:--:--    16
100    54    0    30  100    24      7      5  0:00:04  0:00:04 --:--:--    12
100    54    0    30  100    24      5      4  0:00:06  0:00:05  0:00:01     0
100   883    0   859  100    24    159      4  0:00:06  0:00:05  0:00:01   198


#### Загрузка векторизатора [paraphrase-multilingual-minilm](https://ollama.com/nextfire/paraphrase-multilingual-minilm)

In [4]:
!curl http://localhost:11437/api/pull -d "{\"name\": \"nextfire/paraphrase-multilingual-minilm\"}"

{"status":"pulling manifest"}
{"status":"pulling 35812201e590","digest":"sha256:35812201e590e2f5265c73f70c9a8a0e6ad951317872d66787d9a987bd919e6b","total":120942208,"completed":120942208}
{"status":"pulling cfc7749b96f6","digest":"sha256:cfc7749b96f63bd31c3c42b5c471bf756814053e847c10f3eb003417bc523d30","total":11358,"completed":11358}
{"status":"pulling e700587efa15","digest":"sha256:e700587efa156750d5e99d3336a27efcdf4687573351fa4f100a3d9022220361","total":16,"completed":16}
{"status":"pulling d643d3fca3bf","digest":"sha256:d643d3fca3bf1ec11f2749b1f70be3048b8356c4a2c9bce253fadd28f8d09894","total":409,"completed":409}
{"status":"verifying sha256 digest"}
{"status":"writing manifest"}
{"status":"success"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    81    0    30  100    51     24     41  0:00:01  0:00:01 --:--:--    66
100   763    0   712  100    51    558     39  0:00:01  0:00:01 --:--:--   598


#### Проверка наличия моделей в ollama

In [5]:
!curl -X GET http://localhost:11437/v1/models

{"object":"list","data":[{"id":"nextfire/paraphrase-multilingual-minilm:latest","object":"model","created":1736157579,"owned_by":"nextfire"},{"id":"qwen2.5:0.5b","object":"model","created":1736157578,"owned_by":"library"}]}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   224  100   224    0     0   7115      0 --:--:-- --:--:-- --:--:--  7225


#### Проверка моделей

In [6]:
!curl -X POST http://localhost:11437/api/generate -d "{\"model\": \"qwen2.5:0.5b\", \"prompt\": \"Why is the sky blue?\", \"stream\": false }"

{"model":"qwen2.5:0.5b","created_at":"2025-01-06T09:59:45.656104507Z","response":"The sky is typically blue because it reflects light that falls on it from the ground. The Earth's atmosphere consists of air and particles suspended in it, which refract and scatter light into different colors. When sunlight enters the atmosphere, some of the shorter wavelength blue wavelengths are scattered and reflected by tiny water droplets or ice crystals in the air, while longer wavelength red and yellow wavelengths are absorbed.\n\nThis scattering of light also causes the sky to appear blue because the Earth's surface is dark, reflecting a large portion of the sky as blue. Therefore, the sky appears to be composed primarily of blue colors when viewed from ground level, but in fact it has many shades of blue that can vary depending on the time of day and other atmospheric conditions.","done":true,"done_reason":"stop","context":[151644,8948,198,2610,525,1207,16948,11,3465,553,54364,14817,13,1446,525,

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    77    0     0  100    77      0     63  0:00:01  0:00:01 --:--:--    63
100    77    0     0  100    77      0     34  0:00:02  0:00:02 --:--:--    34
100    77    0     0  100    77      0     23  0:00:03  0:00:03 --:--:--    23
100    77    0     0  100    77      0     18  0:00:04  0:00:04 --:--:--    18
100    77    0     0  100    77      0     14  0:00:05  0:00:05 --:--:--    14
100  2042  100  1965  100    77    352     13  0:00:05  0:00:05 --:--:--   451


In [7]:
!curl http://localhost:11437/api/embeddings -d "{ \"model\": \"nextfire/paraphrase-multilingual-minilm\", \"prompt\": \"Llamas are members of the camelid family\"}"

{"embedding":[0.10243025422096252,0.1631552129983902,-0.0017925690626725554,0.019952569156885147,-0.40203267335891724,-0.25778889656066895,0.47496065497398376,0.0986882895231247,0.15451876819133759,0.010218325071036816,0.236474871635437,-0.6180628538131714,0.34798306226730347,0.05890214815735817,0.20767325162887573,-0.10691619664430618,-0.06268472224473953,0.004951045848429203,-0.2662738859653473,-0.10864783823490143,0.12152256071567535,-0.2818492352962494,-0.21320945024490356,-0.026707563549280167,0.06072978302836418,-0.15678583085536957,-0.13536374270915985,0.10134082287549973,0.22149130702018738,-0.029317112639546394,0.2582590579986572,0.17505118250846863,0.11067056655883789,0.227215975522995,0.07117859274148941,0.2038756161928177,0.008976750075817108,-0.11290179193019867,0.13869039714336395,0.1382545828819275,-0.30882781744003296,-0.014258010312914848,0.1120598092675209,0.46466097235679626,0.03717806935310364,0.1566106528043747,-0.05410315468907356,-0.11345037817955017,0.3638498783

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  7957    0  7850  100   107   7008     95  0:00:01  0:00:01 --:--:--  7104
100  7957    0  7850  100   107   7007     95  0:00:01  0:00:01 --:--:--  7104


### Загрузка и предподготовка данных

Тут у нас есть два пути
Мы можем использовать *LangChain* и *LlamaIndex*
В нашем случае - мы будем использовать *LangChain*, тк проект учебный и рассматриваемый фреймворк представляет нам возможность быстро получить необходимый результат

***! Важная ремарка***:
LlamaIndex позволяет более тонко настроить индексацию и поиск по векторной базе данных

In [1]:
# Импорт библиотек
import os
import re
import pandas as pd
import weaviate
from PyPDF2 import PdfReader

from typing import List
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter

c:\Users\Asus\Documents\GitHub\llamaindex_RAG_MLLM\.venv\Lib\site-packages\PyPDF2\__init__.py:21: DeprecationWarning: PyPDF2 is deprecated. Please move to the pypdf library instead.
  warnings.warn(


In [2]:
# Путь к папке с PDF файлами
DATA_DIR = "./data"
# Подключение к weaviate
client = weaviate.connect_to_local(port=8083, grpc_port=50051)

In [3]:
# Функция для чтения файлов с директории
def read_pdfs_from_folder(folder_path:str = "./data") -> pd.DataFrame:
    """Функция которая читает данные с выбранной директории

    Args:
        folder_path (str): Директория содержащая файлы в формате .pdf

    Returns:
        DataFrame: Прочтённые файлы
    """
    
    pdf_texts = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.pdf'):
            file_path = os.path.join(folder_path, file_name)
            reader = PdfReader(file_path)
            text = ''
            for page in reader.pages:
                text += page.extract_text()
            pdf_texts.append({'file_name': file_name, 'text': text})
    # return pdf_texts
    return pd.DataFrame(pdf_texts)

In [4]:
# Прочитаем данные из папки
data = read_pdfs_from_folder(DATA_DIR)
# # Сохраним данные в формате parquet
# pd.DataFrame(data).to_parquet(os.path.join(DATA_DIR, "data.parquet"))

data.head()

,file_name,text
0,Rebecca_Elliott_-_Painless_Grammar_1997.pdf,cover next page > \n \ntitle:\nauthor:\npublis...
1,А-05м-23 Гольцов МН КП_v1.pdf,Федеральное государственное бюджетное образова...
2,Лабораторная работа_Руководство FPTL.pdf,Федеральное государственное бюджетное образова...


In [5]:
def create_documents(data: pd.DataFrame) -> List[Document]:
    docs = [
        Document(
            page_content=row["text"],
            metadata={
                "doc_id": row["file_name"],
            }
        )
        for _, row in data.iterrows()
    ]
    return docs

In [ ]:
# Читаем файлы с помощью langchain_core.documents
docs = create_documents(data=data)

docs

#### Предобработка документов

У нас есть два варианта как можно предобработать документы
 - Предобработать до разбиения
 - Предобработать после разбиения

Рассмотрим оба варианта, их плюсы и минусы:

##### Предобработка ДО разбиения:
Это значит, что мы сначала 'чистим' или изменяем текст документа, а потом делим его на кусочки, для обработки  

Предобработка включает такие шаги:
 - удаление лишних пробелов, символов, HTML-тегов
 - приведение текста к нижнему регистру (например: 'Привет' -> 'привет')
 - удаление стоп-слов (например: 'и', 'в', 'на')
 - замена сокращений (например: 'т.к.' -> 'так как')
 - исправление опечаток 

**Плюсы**  
1) *Еднообразие текста*: Все части текста будут обработаны одинаково, и мы избежим несоответствий  
2) *Экономия времени*: Предобработка выполняется один раз для всего документа, а не для каждого кусочка  
3) *Упрощение анализа*: Если текст нужно лемматизировать (привести слова к начальной форме), это легче сделать до разбиения  

**Минусы**  
1) *Проблемы с контекстом*: Предобработка может удалить важные слова, а это может исказить смысл  
2) *Невозможность адаптации*: Если разные куски текста требуюи разной обработки общий подход может не подойти  

##### Предобработка ПОСЛЕ разбиения:
Здесь мы сначала делим текст на кусочки, а потом каждый кусочек обрабатываем отдельно   

**Плюсы**  
1) *Гибкость*: Можно применять разные подходы к разным кусочкам  
***Пример -> Для технического текста можно удалить формулы, а для описания - оставить***
2) *Меньше ошибок с контекстом*: Разбиение сохраняет структуру текста, и мы можем обрабатывать кусочки, учитывая их содержимое  
***Пример -> Если в одном кусочке есть дата, а в другом - событие, мы обработаем их отдельно и не потеряем смысл***  
3) *Локальная оптимизация*: Можно обрабатывать только те кусочки, которые важны для задачи  
***Пример -> Если часть текста не нужна, её можно исключить***  

**Минусы**  
1) *Больше ресурсов*: Каждый кусочек обрабатывается отдельно, что может занять больше времени и памяти  
2) *Потеря глобального контекста*: Если предобработка зависит от всей структуры текста, разбиение может её нарушить  

##### Пример для наглядности

Приведём приммер исходного текста - "Компания Apple представила новый iPhone. Цена на него составит $999. Это вызвало бурные обсуждения в социальных сетях."

**Предобработка ДО разбиения**  
1) Удаляем знаки пунктуации  
        *"Компания Apple представила новый iPhone Цена на него составит 999 Это вызвало бурные обсуждения в социальных сетях"*  
2) Приводим к нижнему регистру  
        *"компания apple представила новый iphone цена на него составит 999 это вызвало бурные обсуждения в социальных сетях"*  

3) Разбиваем на кусочки  
 - *"компания apple представила новый iphone"*   
 - *"цена на него составит 999"*  
 - *"это вызвало бурные обсуждения в социальных сетях"*  

**Предобработка ПОСЛЕ разбиения**
1) Сначала делим текст на кусочки
 - "Компания Apple представила новый iPhone."
 - "Цена на него составит $999."
 - "Это вызвало бурные обсуждения в социальных сетях."
2) Обрабатываем каждый кусочек
 - "компания apple представила новый iphone"
 - "цена на него составит 999"
 - "это вызвало бурные обсуждения в социальных сетях"


**Вывод**  
 - Если текст гомогенный (однотипный), например, техническая документация, то лучше предобрабатывать ДО разбиения  
 - Если текст разнообразный (содержит разные стили и форматы), то предобработка ПОСЛЕ разбиения даст больше гибкости  

*Для RAG-системы часто используется предобработка после разбиения, чтобы сохранить максимальный контекст в каждом кусочке*

#### Разбиение документов на куски

In [7]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [8]:
# В нашем случае тексты разбиты на логические блоки с помощью \n
# Но символ \n встречается очень часто, и разбивать текст по ним не имеет смысл
# Причиной этому является - слишком маленькие блоки получающиеся в результате разбиения, в дальнеёшем нам будет сложно искать и отвечать

def split_document(docs: List[Document],
                   chunk_size: int = 512, # 512
                   chunk_overlap: int = 256, # 256 
                   is_separator_regex: bool = False # 
                   ) -> List[Document]:
    """
    Разбивает текст документов на небольшие части для удобства обработки, поиска и анализа.

    Args:
        docs (List[Document]): Список объектов `Document`, содержащих текст и метаданные.
        chunk_size (int): Максимальный размер одного фрагмента текста (в символах). 
                          По умолчанию 512.
        chunk_overlap (int): Количество символов, которые перекрываются между соседними фрагментами.
                             Это помогает сохранить контекст между частями. По умолчанию 256.
        is_separator_regex (bool): Указывает, является ли разделитель регулярным выражением.
                                   Если False, используется обычный строковый разделитель. По умолчанию False.

    Returns:
        List[Document]: Список объектов `Document`, содержащих нарезанные части текста.
                        Каждая часть сохраняет оригинальные метаданные.

    Пример использования:
        documents = [
            Document(page_content="Большой текст, который нужно разбить", metadata={"doc_id": "1"})
        ]
        split_docs = split_document(documents, chunk_size=256, chunk_overlap=128)
    """
    splitter = CharacterTextSplitter(
        # Рассматриваемые тексты имеют разную структуру и мы не можем корректно указать какой из разделителей нужно использовать
        # Поэтому попробуем просто разделить текст по символам
        separator='',                      
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        is_separator_regex=is_separator_regex,
    )
    return splitter.split_documents(docs)

In [ ]:
# Проверяем исходные данные
print(f"Исходное количество документов: {len(docs)}")
for i, doc in enumerate(docs):
    print(f"Документ {i}: длина текста = {len(doc.page_content)} символов")

split_docs = split_document(docs=docs)
# Проверяем результат
print(f"Количество кусков после разбиения: {len(split_docs)}")
for i, doc in enumerate(split_docs):
    # print(f"Кусок {i}: {doc.page_content[:50]}... (длина: {len(doc.page_content)})")
    print(f"Кусок {i}: длина куска: {len(doc.page_content)})")

# Исходное количество документов: 3
# Документ 0: длина текста = 308153 символов
# Документ 1: длина текста = 12084 символов
# Документ 2: длина текста = 55970 символов
# Количество кусков после разбиения: 1468

#### Постобработка документов

Максимально распишем следующую ячейку:

Мы будем использовать модуль re (регулярные выражения)

- Создаем реглярное выражение для поиска последовательнсотей символов
    - \n - перевод строки  
    - \t - табуляции  
    - \r - возврат каретки  
```python
    TABS_REGEXP_TEXT = r'[\n\t\r]+'
```

- Создаем реглярное выражение для поиска HTML-тегов

```python
    TAGS_REGEXP_TEXT = r'(\<(/?[^>]+)>)'
```

- Создаем регулярное выражение для поиска всех символов, которые не являются латинскими буквами (a-z), кириллическими буквами (а-я) или цифрами (0-9)

```python
    SYMBOLS_REGEXP_TEXT = r'[^a-zа-я0-9]'
    SYMBOLS_WITHDASH_REGEXP_TEXT = r'(\s\-\s)|(\-\s)|(\s\-)|([^a-zа-я0-9\-])'
```

- Создаем регулярное выражение для поиска последовательностей из одного или более пробельных символов 

```python
    SPACES_REGEXP_TEXT = r"\s+"
```


In [10]:
# import re

TABS_REGEXP_TEXT = r'[\n\t\r]+'
TAGS_REGEXP_TEXT = r'(\<(/?[^>]+)>)'
SYMBOLS_REGEXP_TEXT = r'[^a-zа-я0-9]'
SYMBOLS_WITHDASH_REGEXP_TEXT = r'(\s\-\s)|(\-\s)|(\s\-)|([^a-zа-я0-9\-])'
SPACES_REGEXP_TEXT = r"\s+"

TABS_REGEXP = re.compile(TABS_REGEXP_TEXT)
TAGS_REGEXP = re.compile(TAGS_REGEXP_TEXT)
SYMBOLS_REGEXP = re.compile(SYMBOLS_REGEXP_TEXT)
SYMBOLS_WITHDASH_REGEXP = re.compile(SYMBOLS_WITHDASH_REGEXP_TEXT)
SPACES_REGEXP = re.compile(SPACES_REGEXP_TEXT)


def text_clean(text: str, keep_dash: bool = False) -> str:
    """Очистка текста

    Args:
        text (str): Исходный текст
        keep_dash (bool, optional): Определяем, нужно ли сохранить дефисы в тексте

    Returns:
        str: Очищенную строку
    """
    result = str(text)
    result = result.lower()
    result = result.replace("ё", "е")
    result = TABS_REGEXP.sub(" ", result)
    result = TAGS_REGEXP.sub(" ", result)
    if keep_dash:
        result = SYMBOLS_WITHDASH_REGEXP.sub(" ", result)
    else:
        result = SYMBOLS_REGEXP.sub(" ", result)
    result = result.strip()
    result = SPACES_REGEXP.sub(" ", result)    
    return result

def postprocess(docs: List[Document], keep_dash: bool = False) -> List[Document]:
    """Постобработка кусков текста

    Args:
        docs (List[Document]): Нарезанныен куски текста
        keep_dash (bool, optional): Определяем, нужно ли сохранить дефисы в тексте

    Returns:
        List[Document]: Очищенные куски текста
    """
    for doc in docs:
        doc.page_content = text_clean(doc.page_content)
    return docs

In [11]:
split_docs = postprocess(split_docs)

split_docs[:2]
# split_docs[1300:1302]

[Document(metadata={'doc_id': 'Rebecca_Elliott_-_Painless_Grammar_1997.pdf'}, page_content='cover next page title author publisher isbn10 asin print isbn13 ebook isbn13 language subject publication date lcc ddc subject cover next page page i painless grammar rebecca elliott ph d illustrated by laurie hamilton page ii copyright 1997 by rebecca elliott illustrations copyright 1997 by barron s educational series inc all rights reserved no part of this book'),
 Document(metadata={'doc_id': 'Rebecca_Elliott_-_Painless_Grammar_1997.pdf'}, page_content='trated by laurie hamilton page ii copyright 1997 by rebecca elliott illustrations copyright 1997 by barron s educational series inc all rights reserved no part of this book may be reproduced in any form by photostat microfilm xerography or any other means or incorporated into any information retrieval system electronic or m echanical without the written permission of the copyright owner all inquiries should be add')]

### Векторизация

Векторизация основывается на преобразовании объектов в многомерное пространство. В этом пространстве каждый объект представляется вектором, а расстояние между векторами отражает степень их сходства.

Так, нам необходимо с помощью эмбедера получить векторное представление для загруженных документов. Поскольку в будущем мы будем осуществлять поиск по близости с помощью получившихся векторов.

In [17]:
import numpy as np

from llama_index.core import VectorStoreIndex
# from llama_index.embeddings.ollama import OllamaEmbedding
# from langchain_ollama import OllamaEmbeddings
from langchain_ollama.embeddings import OllamaEmbeddings

In [24]:
def vectorize(docs:List[Document],
              host: str = "http://localhost:11437", 
              model_name: str = "nextfire/paraphrase-multilingual-minilm", 
              path_to_vectors: str = "data/embeddings.npy"
              ) -> List[List[float]]:
    """Генерирует эмбеддинги из входных документов и сохраняет их в файл

    Args:
        docs (List[Document]): Список документов
        host (str): Адрес на котором работает векторизатор
        model_name (str): Название модели в нашем случае -- "nextfire/paraphrase-multilingual-minilm".
        path_to_vectors (str): Путь для сохранения эмбеддингов

    Returns:
        List[List[float]]: Список эмбеддингов, где каждый эмбеддинг соответствует одному документу
    """
        
    embed_model = OllamaEmbeddings(model=model_name, base_url=host)
    # embed_model = OllamaEmbedding(model_name=model_name, base_url=host)
    
    embeddings = embed_model.embed_documents(
    # embeddings = embed_model.get_text_embedding_batch(
        [
            doc.page_content for doc in docs
        ]
    )
    np.save(
        file=path_to_vectors,
        arr=embeddings
    )
    return embeddings

In [26]:
# Обрабатываем наши тексты
embeddings_docs = vectorize(docs=split_docs)

In [27]:
len(embeddings_docs)

1468

### Настройка поискового движка

Поисковой движок в RAG — это компонент, отвечающий за поиск релевантной информации в базе данных или хранилище знаний, чтобы дополнить генеративную модель контекстом для точного ответа


Мы будем использовать два типа поиска:
- Векторный -- *Ищет документы на основе их смыслового сходства. Запрос и документы преобразуются в векторы, а затем сравниваются по близости (например, с помощью косинусного расстояния). Используется для поиска по смыслу*


- Полнотекстовый -- *Ищет документы по ключевым словам и фразам, учитывая точное совпадение текста. Подходит для поиска конкретных терминов или выражений*

In [28]:
# import weaviate
import weaviate.classes.config as wc
from bm25s.stopwords import STOPWORDS_RUSSIAN
from weaviate.collections.classes.config import StopwordsPreset

resource module not available on Windows


c:\Users\Asus\Documents\GitHub\llamaindex_RAG_MLLM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
def setup_search_engine(
    port: str = '8083',
    grpc_port: str = '50051',
    
    mapping_types = {
        "doc_id": wc.DataType.TEXT,  # Уникальный идентификатор документа
        "text": wc.DataType.TEXT     # Основной текст документа
    }
) -> bool:
    """Настраивает поисковый движок с использованием Weaviate, 
    создавая коллекцию для хранения и поиска документов

    Args:
        port (str): Порт для подключения к локальному серверу Weaviate
        grpc_port (str): gRPC-порт для подключения к серверу Weaviate
        mapping_types (dict): Словарь, описывающий структуру данных коллекции. 
    Ключи — имена полей, значения — типы данных (например, wc.DataType.TEXT).

    Returns:
        bool: Возвращает True, если коллекция успешно создана, и False в случае ошибки
    """
    
    # Создаём подключение
    with weaviate.connect_to_local(
       port=port, 
       grpc_port=grpc_port
       ) as client:
       
        client.collections.delete_all()
        collection = client.collections.create(
            name='load_data_from_file',
            properties=[
                wc.Property(
                    name=column_name,
                    data_type=mapping_types[column_name]
                    ) for column_name in mapping_types
            ],
            
            vectorizer_config=wc.Configure.Vectorizer.none(),
            # Настройка hnsw
            vector_index_config=wc.Configure.VectorIndex.hnsw(
                distance_metric=wc.VectorDistances.COSINE,
                ef_construction=256,    
                max_connections=128,    
                quantizer=wc.Configure.VectorIndex.Quantizer.bq(), 
                ef=-1,                  
                dynamic_ef_factor=15,   
                dynamic_ef_min=200,     
                dynamic_ef_max=1000 
            ),
            # Настройка полнотекстового поиска
            inverted_index_config=wc.Configure.inverted_index(
                stopwords_additions=list(STOPWORDS_RUSSIAN),
                stopwords_preset=StopwordsPreset.NONE
            )
        )
        
setup_search_engine()

Разберём приведённый код поподробнее:

**Блок кода 1**  

```python
wc.Configure.Vectorizer.none()
```

Указывает, что векторизация данных не будет выполняться на стороне Weaviate

**Блок кода 2**  

Настройка HNSW (Hierarchical Navigable Small World) индекса для векторного поиска. Этот индекс используется для быстрого поиска ближайших соседей 

![HNSW](img/HNSW.svg)

```python
vector_index_config=wc.Configure.VectorIndex.hnsw(
                distance_metric=wc.VectorDistances.COSINE,
                ef_construction=256,    
                max_connections=128,    
                quantizer=wc.Configure.VectorIndex.Quantizer.bq(), 
                ef=-1,                  
                dynamic_ef_factor=15,   
                dynamic_ef_min=200,     
                dynamic_ef_max=1000 
            )
```
В этой реализации мы указываем следущие параметры:

- Явно указываем метрику расстояния для поиска
- Размер динамического списка ближайших соседей при построении индекса. *(Большие значения повышают точность, но увеличивают время построения ef_construction)*
- Максимальное кол-во связей для каждой вершины в графе. *(Чем больше связей, тем ваше точность, но тем больше памяти потребуется max_connections)*
- Размер динамического списка при поиске. *(В нашем случае -- автоматическое определение размера ef)*


**Блок кода 3**

Настройка полнотекстового поиска и использованием инвертированного индекса

```python
inverted_index_config=wc.Configure.inverted_index(
                stopwords_additions=list(STOPWORDS_RUSSIAN),
                stopwords_preset=StopwordsPreset.NONE
            )
```

Параметры:

- stopwords_additions -- Дополнительные стоп-слова на русском языке, которые будут исключены из индексации (Дополнительно указываем, что будем использовать только пользовательские стоп-слова)


In [30]:
# !curl http://localhost:8083/v1/schema

## Сохранение индекса

In [31]:
from typing import List
from weaviate.util import generate_uuid5

In [32]:
def fill_weaviate(
    docs: List[Document], 
    embeddings: List[List[float]],
    port: str = '8083',
    grpc_port: str = '50051',
    collection_name: str = 'load_data_from_file',
) -> None:
    
    # Создаём подключение
   with weaviate.connect_to_local(
       port=port, 
       grpc_port=grpc_port
       ) as client:
       
       collection = client.collections.get(collection_name)
       with collection.batch.dynamic() as batch:
           for doc, embedding in zip(docs, embeddings):
               data = {
                   'text': doc.page_content,
                   'doc_id': doc.metadata['doc_id']
               }
               obj_uuid = generate_uuid5(doc.metadata["doc_id"])
               
               batch.add_object(
                    properties=data,
                    uuid=obj_uuid,
                    vector=embedding
               )
               
               if len(collection.batch.failed_objects) > 0:
                   raise RuntimeError('Вставка объектов закончилась неудачей((')

In [33]:
fill_weaviate(
    docs=split_docs,
    embeddings=embeddings_docs
)

c:\Users\Asus\Documents\GitHub\llamaindex_RAG_MLLM\.venv\Lib\site-packages\pydantic\main.py:214: ResourceWarning: unclosed <socket.socket fd=5260, family=23, type=1, proto=0, laddr=('::1', 59502, 0, 0), raddr=('::1', 11437, 0, 0)>
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


# Запуск RAG локально (Бой)

Нам осталось несколько шагов для построения полной RAG системы

In [69]:
from langchain_ollama import OllamaLLM
# from llama_index.llms.ollama import Ollama

In [70]:
# Вспомним функию для очистки текста
text_clean('ПриВет мИр !!! _!')

'привет мир'

Создаём клиент для работы с LLM

In [71]:
llm = OllamaLLM(
       model='qwen2.5:0.5b',
       base_url="http://localhost:11437",
)

# Напишем запрос к модели
prompt = """
Привет! Почему Россия лучшая страна?
"""

# mesage = llm.complete(prompt)

print(llm.invoke(prompt))

Россия является одним из самых значимых и важных государств в мире на несколько причин:

1. Власть: Россия имеет мощное военное государство, которое отвечает за все государственные задачи, от защиты страны до обеспечения безопасности.

2. Страны-военщины: Москва обладает огромным количеством военнослужащих и боевых действующих, что позволяет выполнять важные военные операции.

3. Влияние на мир: Россию часто считают важной для мира и безопасности, особенно во время военных или внутренних экономических разногласий.

4. Культура: Россия известна своим богатым творческим импровизированием и культурой.

5. Величия: Москва занимает важную позицию во многих международных организациях.

6. История: Россия имеет долгую историю как национальной, так и культурного, и этот опыт может быть достоверно полезен для других стран.

7. Заказы: Москва и другие страны-производители товаров и услуг часто приглашают экспорт, что приводит к глобальному влиянию на экономику и политику многих стран мира.

8. С

Создаём клиент для работы с векторизацией

In [40]:
vectorizer = OllamaEmbeddings(
# vectorizer = OllamaEmbedding(
    model='nextfire/paraphrase-multilingual-minilm',
    base_url="http://localhost:11437",
)


print(list(vectorizer.embed_query("What is the meaning of life?")))
# print(list(vectorizer.get_general_text_embedding("What is the meaning of life?")))

[0.024333974, 0.027174825, -0.0067018764, 0.014657992, -0.095907025, -0.085999474, 0.13963348, 0.01984035, -0.010825936, 0.027281752, 0.07512576, -0.14539377, 0.022449352, -0.02175226, 0.07165339, -0.022190709, -0.0020747203, -0.098937556, -0.051401973, -0.024702994, 0.002016932, -0.026661372, -0.018259509, -0.018064039, 0.031074664, -0.02545282, -0.08748126, -0.004877896, 0.08325033, 0.008110117, 0.05884749, 0.024842786, 0.05521977, 0.04374893, 0.004295327, 0.06872931, 0.024665283, -0.016006663, -0.019855365, -0.026829446, -0.056481075, 0.049340934, 0.041310415, 0.026991453, -0.001221893, -0.028030988, -0.019083964, -0.017961057, 0.027202941, -0.02017694, -0.016473884, -0.07783997, -0.057090897, -0.0043340763, 0.06539574, -0.02469487, 0.040410683, 0.015415137, -0.049653236, 0.0234235, 0.028451215, -0.042905852, -0.12578513, 0.021486547, -0.023956237, 0.05465443, -0.032857552, 0.0455701, 0.022862364, 0.012086024, 0.0065209917, -0.05922973, -0.050160006, 0.077816814, 0.0079791555, -0.03

Мы будем сохранять промпты в созданную папку - prompts

In [65]:
from langchain_core.prompts import PromptTemplate

# Создаем директорию для хранения файлов с шаблонами, если она не существует
os.makedirs('prompts', exist_ok=True)

In [66]:
def get_prompt(
    path_to_prompt_file: str,
    in_vars: List[str]
)-> PromptTemplate:
    """Загружает шаблон из файла

    Args:
        path_to_prompt_file (str): Путь к файлу
        in_vars (List[str]): Список перменных используемых в шаблоне

    """
    return PromptTemplate.from_file(
        template_file=path_to_prompt_file,
        input_variables=in_vars,
        encoding='UTF-8'
    )

## Создаём ретривер

In [43]:
from langchain_core.documents import Document
from langchain_weaviate.vectorstores import WeaviateVectorStore
from langchain_core.vectorstores.base import VectorStoreRetriever

In [44]:
# # from langchain_weaviate.vectorstores import WeaviateVectorStore
# # from langchain_core.vectorstores.base import VectorStoreRetriever


# from llama_index.vector_stores.weaviate import WeaviateVectorStore
# from llama_index.embeddings.ollama import OllamaEmbedding
# # from llama_index.query_engine import RetrieverQueryEngine

In [45]:
def create_retriever(
    port: str = '8083',
    grpc_port: str = '50051',
    collection_name: str = 'load_data_from_file',
    text_field_for_search: str = 'text',
    embeddings_model = vectorizer,
    attributes: List[str] = ['doc_id', 'text']
) -> VectorStoreRetriever:
    """Создает ретривер на основе LlamaIndex с Weaviate векторным хранилищем
    """
    
    # Подключение к Weaviate
    weaviate_client = weaviate.connect_to_local(
        port=port,
        grpc_port=grpc_port,
    )
    
    # Создаем WeaviateVectorStore
    vector_store  = WeaviateVectorStore(
        client=weaviate_client,
        index_name=collection_name,
        text_key=text_field_for_search,
        attributes=attributes,
        embedding=embeddings_model
    )
    
    # Создаем ретривер на основе векторного хранилища
    retriever = vector_store.as_retriever(
       search_type="similarity",
       search_kwargs={'k': 3}
    )
    
    # # Создаем QueryEngine для выполнения запросов
    # query_engine = RetrieverQueryEngine(retriever=retriever)
    
    return retriever

In [53]:
retriever = create_retriever()

retriever.invoke('Расскажи про модель опционов Блека-Шоулза')

[Document(metadata={'doc_id': 'А-05м-23 Гольцов МН КП_v1.pdf'}, page_content='ло полезным инструментом для принятия обоснованных инвестиционных решений в условиях высокой волатильности рынка таким образом результаты данной работы подчеркивают важность адаптивного подхода к моделированию опционов и его применения в практике финансового анализа и управления рисками'),
 Document(metadata={'doc_id': 'Rebecca_Elliott_-_Painless_Grammar_1997.pdf'}, page_content='25 regular 32 split infinitives 26 subject agreement 155163 subjunctive mood 3031 tenses 2325 voice active versus passive 2628 w waiting on for 196 weights numbers 139 well good 186 which that 196198 which who 198 who whom 198199 why how come what for 199200 words omitted 219220 commas 91 overused 221222 selection 250252 unnecessary 215219 247 usage 173210 xyz you 21 previous page page 264'),
 Document(metadata={'doc_id': 'Лабораторная работа_Руководство FPTL.pdf'}, page_content='1 c nil c nil 1 c cons args 1 3 fpredicate 1 2 3 filte

# RAG

In [56]:
from typing import Callable
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

Созадаим цепочку процессов которые будут формировать вывод к нашему приложению

In [72]:
def chain(
    llm: Ollama,
    retriever: VectorStoreRetriever,
    format_text: Callable,
    prompt: PromptTemplate
) -> str:
    return (
        {'context': retriever, 'query': RunnablePassthrough() | format_text}
        | prompt
        | llm
        | StrOutputParser()
    )


rag_chain = chain(
    llm=llm,
    retriever=retriever,
    format_text=text_clean,
    prompt = get_prompt(
        path_to_prompt_file='prompts/ollama_prompt.txt',
        in_vars=['query', 'context']
    )        
)

C:\Users\Asus\AppData\Local\Temp\ipykernel_18744\1161606906.py:12: DeprecationWarning: `input_variables' is deprecated and ignored.
  return PromptTemplate.from_file(


In [78]:
for chunk in rag_chain.stream("Что ты знаешь о FTPL"):
    print(chunk, end="", flush=True)

Контекст содержит информацию о ф-ции FPTL, которая является частью стандартной пакетной инструментарии Python. Ф-ция FPTL используется для быстрой и удобной построения и отслеживания опорных элементов в списке.

В контексте документа 1 "Лабораторная работа_Руководство FPTL.pdf", мы видим, что ф-ция FPTL предназначена для быстрого и эффективного отслеживания опорных элементов в списке. Она также предоставляет функции для подсчета количества элементов в списке и обновления списка опорных элементов.

В контексте документа 2 "Rebecca_Elliott_-_Painless_Grammar_1997.pdf", мы видим, что ф-ция FPTL используется для быстрой построения и отслеживания опорных элементов в списке. Она позволяет быстро находить первый независимый элемент списка.

В контексте документа 3 "Лабораторная работа_Руководство FPTL.pdf", мы видим, что ф-ция FPTL используется для быстрой и эффективной построения и отслеживания опорных элементов в списке. Она также предоставляет функции для подсчета количества элементов в сп

In [81]:
for chunk in rag_chain.stream("В FPTL есть возможность использования функционалов - функций, принимающих в качестве одного из параметров другую функцию"):
    print(chunk, end="", flush=True)

Да, в FPTL (Фондовые Техники и Принятие Оптимизированных решения) есть возможности использования функций принимающих один из параметров другого функции. Эти функции обычно называются "функционалами" или "представлениями", которые позволяют вычислять значения или результаты в зависимости от определенных условий.

В Document(metadata={'doc_id': 'Лабораторная работа_Руководство FPTL.pdf'}, page_content='1 c nil c nil 1 c cons args 1 3 fpredicate 1 2 3 filter c cons 2 3 filter args 1 c cons 2') можно прочитать, что в FPTL руководстве приведены правила, которые позволяют определить, какая функция будет работать с инструментом. Например, если у инструмента есть функция `a`, то она может быть использована для расчета значения `b` из функции `f`. Аналогично, если у инструмента есть функция `x`, то она может быть использована для расчета значения `y` из функции `g`.

Эти функции позволяют использовать различные подходы и агрегатные методы в работе с данными, что позволяет вычислять результаты э

# Выводы

Какие выводы можо сделать по проведённой работе?)

Я думаю итог можно поделить на 2 типа:

1 - Положительный 
- Я построил RAG систему 😄
- Получил опыт работы с наиболее популярными библиотеками для работы с LLM (langchain, llama_index)
- Интегрировал и настроил векторную базу данных (Weaviate)
- Поработал с моделью - Qwen-2.5

2 - То на что стоит обратить внимание
- В приведённом коде достаточно, много всяких рычажков, которые можно покрутить))
    - Настройки llm (температура, top-k, top-p)
    - Настройки поисковика (Дополнительно опробовать алгоритмы поиска, связанные с деревьями, хешированием)
    - Настроить и покрутить промпт

- В потенциальном развитии возможно расширить RAG добавив:
    - Классификатор вопросов
    - Маршрутизатор
    - Агентную систему
    - Добавить возможность сверять вопрос и ответ с помощью Cross-Encoder или Bi-Encoder

*Часть из этих затей планируется опробовать в других проектах*